In [1]:
import numpy as np

In [14]:
def b2(T, sig=1.0, eps=1.0, verbose=True):
    """Returns second virial coefficient of Lennard-Jones fluid
    
    Parameters
    ----------
    T : float
        Absolute temperature (K)
    sig : float
        Size parameter (Angstrom)
    eps : float
        Energy parameter (K) ##(kJ/mol)
        
    Returns
    -------
    b2 : float
        second virial coefficient (cm^3/mol)
    """
    
    # Reduced temperature
    #Tr = T*8.314472/(eps*1000.0)
    Tr = T/eps
    
    # Check if the reduced temperature is safe range
    if verbose and (Tr < 0.3 or Tr > 10000.0):
        print(f'Temperature {Tr} K is not in range 298-3000 K.')
        
    # Reduced virial coefficient
    Br  = 1.9863*Tr**(-0.25) - 1.5985*Tr**(-0.5) - 6.7636*Tr**(-1.5)
    Br += 8.0952*Tr**(-2.0) - 4.8797*Tr**(-2.5) + 0.89417*Tr**(-3.5)
    Br -= 0.25573*Tr**(-4.0) - 0.015386*Tr**(-5.0)
    
    Na = 6.0221409e23
    b0 = 2*np.pi*Na*(sig*1e-8)**3/3
    
    B2 = Br*b0
    
    return B2

In [27]:
b2(300, 3.61, 190.0)

-63.99906747504426

In [106]:
class Omega:
    """Collision integrals for Lennard-Jones potential"""
    
    def __init__(self, coeffs):
        if isinstance(coeffs, dict):
            self.coeffs = coeffs
        elif isinstance(coeffs, np.ndarray):
            self.coeffs = self.make_dict(coeffs)
        else:
            raise TypeError
            
    def make_dict(self, coeffs_array):
        """Creates a dictionary from supplied coefficient ndarray.
        
        Works for a particular structure of ndarray - should be modified
        if other format is used.
        """
        
        ls_tuples = [(1, i) for i in range(1, 8)] + [(2, i) for i in range(2, 7)] + [(3,3), (3,4), (3,5), (4,4)]
        b_list = [1, 3, 5, 8, 10, 6]
        c_list = [2, 4, 7, 9, 11, 12]
        b_fac = np.array([1., 1., 10., 10., 100., 1000.])
        c_fac = np.array([1., 10., 10., 100., 1000., 10000.])

        coeffs = {}
        for i in range(16):
            ls = ls_tuples[i]
            coeffs[ls] = {}
            coeffs[ls]['a'] = coeffs_array[0,i]
            coeffs[ls]['b'] = coeffs_array[b_list, i]/b_fac
            coeffs[ls]['c'] = coeffs_array[c_list, i]/c_fac

        return coeffs
        
    def get(self, Tr, l=1, s=1):
        """Returns collision integral for a given temperature"""
        coeff = self.coeffs[(l, s)]
        b = coeff['b']
        c = coeff['c']
        omega = coeff['a']
        omega += sum([b[k-1]*Tr**(-k) + c[k-1]*np.log(Tr)**k for k in range(1, 7)])
    
        return omega

In [107]:
# read data from file
cc = np.loadtxt('../data/raw/coeffs_num.txt')

In [108]:
omg = Omega(cc)

In [109]:
omg.get(1.0, 2, 6)

1.1127095589290001